In [10]:
import json
import pandas as pd

from DBUpdater import DBUpdater
from crawler import Crawler

In [2]:
def getBaekjoonData(crawler):
    crawler.login_solved()
    df = crawler.read_solved()
    
    return df

In [3]:
def updateDBBaekjoon(dbupdater, df):
    dbupdater.replace_into_db(df)

In [4]:
with open('info.json', 'r') as file:
    data = json.load(file)
    db_pw = data['db_info']['pw']

    id = data['user_info']['id']
    pw = data['user_info']['pw']

In [8]:

crawler = Crawler(id, pw)

In [9]:
df=getBaekjoonData(crawler)

백준 문제 크롤링 완료


In [10]:
len(df)

4543

In [15]:
df[df['level']==3]

,number,title,isSolved,level,attempt
135,1085,직사각형에서 탈출,False,3,41876
136,1247,부호,True,3,3162
137,1267,핸드폰 요금,False,3,6817
138,1284,집 주소,False,3,3491
139,1547,공,False,3,7999
...,...,...,...,...,...
368,26145,출제비 재분배,False,3,385
369,26264,빅데이터? 정보보호!,False,3,582
370,26645,성장의 비약 선택권,False,3,433
371,27159,노 땡스!,False,3,805


In [17]:
df.to_csv('baekjoon_info.csv', encoding='utf8')

In [5]:
df = pd.read_csv('baekjoon_info.csv')

In [5]:

dbupdater = DBUpdater(db_pw)

In [6]:
number, title = dbupdater.select_problem()

c:\Users\Kyeul\Desktop\code\Automation_repo-master\Automation_repo-master\깃헙인증_자동화\DBUpdater.py:101: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.conn)


In [7]:
query = f"""
백준 {number}, '{title}' 파이썬으로 풀어줘.
입력 예시는 작성 하지마.
출력 예시도 작성 하지마.
설명도 작성 하지마.
코드만 작성해줘.
"""

In [8]:
print(query)


백준 1004, '어린 왕자' 파이썬으로 풀어줘.
입력 예시는 작성 하지마.
출력 예시도 작성 하지마.
설명도 작성 하지마.
코드만 작성해줘.



In [9]:

import openai

with open('info.json', 'r') as file:
    data = json.load(file)
    key = data['openai-api']['key']


openai.api_key = key



In [12]:
dbupdater.plus_attempt(number)

1004문제의 시도 횟수: 1


In [10]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{"role":"user", "content":query}]
)
result = response['choices'][0]['message']['content']
print(result)



import math

T = int(input())

for _ in range(T):
    x1, y1, x2, y2 = map(int, input().split())
    n = int(input())
    count = 0
    
    for i in range(n):
        cx, cy, r = map(int, input().split())
        d1 = math.sqrt((cx-x1)**2 + (cy-y1)**2)
        d2 = math.sqrt((cx-x2)**2 + (cy-y2)**2)
        if (d1 < r and d2 > r) or (d1 > r and d2 < r):
            count += 1
    
    print(count)


In [13]:
dbupdater.update_isSolved(number)

f{number} 문제를 해결했습니다.


In [36]:
with open('result.txt', 'w', encoding='utf8') as f:
    f.write(result)

In [1]:
import json

from DBUpdater import DBUpdater
from crawler import Crawler
from run_gpt import RunGPT

In [10]:
def getBaekjoonData(crawler):
    # 솔브드 크롤링
    # 웹에 로그인 한 뒤 데이터를 모은다.
    crawler.login_solved()
    df = crawler.read_solved()
    
    return df

def updateDBBaekjoon(dbupdater, df):
    dbupdater.replace_into_db(df)


def get_solution_baekjoon_with_GPT(dbupdater, runGPT):
    number, title = dbupdater.select_problem() # DB에서 적정 문제 가져오기

    query = f"""
            백준 {number}, '{title}' 파이썬으로 풀어줘.
            입력 예시는 작성 하지마.
            출력 예시도 작성 하지마.
            설명도 작성 하지마.
            코드만 작성해줘.
            """
    result_code = runGPT.run(query)

    with open(f'{number}_solution.py', 'w') as f:
        f.write(result_code)

In [5]:
dbupdater = DBUpdater(db_pw)
runGPT = RunGPT(key)

In [11]:
get_solution_baekjoon_with_GPT(dbupdater, runGPT)

c:\Users\Kyeul\Desktop\code\Automation_repo-master\Automation_repo-master\깃헙인증_자동화\DBUpdater.py:101: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.conn)


In [6]:
import re
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import json

In [1]:
import json

from DBUpdater import DBUpdater
from crawler import Crawler
from run_gpt import RunGPT

In [2]:
with open('info.json', 'r') as file:
    data = json.load(file)
    db_pw = data['db_info']['pw']

    id = data['user_info']['id']
    pw = data['user_info']['pw']

    key = data['openai-api']['key']

In [3]:

dbupdater = DBUpdater(db_pw)
run_gpt = RunGPT(key)

In [4]:
number, title = dbupdater.select_problem()

c:\Users\Kyeul\Desktop\code\Automation_repo-master\Automation_repo-master\깃헙인증_자동화\DBUpdater.py:101: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.conn)


In [5]:
query = f"""
백준 {number}, '{title}' 파이썬으로 풀어줘.
입력 예시는 작성 하지마.
출력 예시도 작성 하지마.
설명도 작성 하지마.
코드만 작성해줘.
"""
print(query)


백준 1016, '제곱 ㄴㄴ 수' 파이썬으로 풀어줘.
입력 예시는 작성 하지마.
출력 예시도 작성 하지마.
설명도 작성 하지마.
코드만 작성해줘.



In [6]:
solution = run_gpt.run(query)
print(solution)

# error case
# RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 34fdb85f219b3b4e768e28bd817ce85a in your message.)



import math

min_, max_ = map(int, input().split())
squares = set([number ** 2 for number in range(2, int(math.sqrt(max_)) + 1)])
result = set(range(min_, max_ + 1))

for sqr in squares:
    number = math.ceil(min_ / sqr) * sqr
    while number <= max_:
        result.discard(number)
        number += sqr
print(len(result))


In [15]:
dbupdater.plus_attempt(number)

1012문제의 시도 횟수: 4


In [6]:
dbupdater.update_isSolved(number)

1016 문제를 해결했습니다.


In [12]:
import json

from DBUpdater import DBUpdater
from crawler import Crawler
from run_gpt import RunGPT

import re

In [2]:
with open('info.json', 'r') as file:
    data = json.load(file)
    db_pw = data['db_info']['pw']

    id = data['user_info']['id']
    pw = data['user_info']['pw']

    key = data['openai-api']['key']

In [3]:

dbupdater = DBUpdater(db_pw)
run_gpt = RunGPT(key)

In [7]:
number, title = dbupdater.select_problem()
print(number, title)

이번에 풀이할 문제는 '1021', '회전하는 큐'입니다.
1021 회전하는 큐


c:\Users\Kyeul\Desktop\code\Automation_repo-master\Automation_repo-master\깃헙인증_자동화\DBUpdater.py:101: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.conn)


In [8]:
query = f"""
백준 {number}, '{title}' 파이썬으로 풀어줘.
입력 예시는 작성 하지마.
출력 예시도 작성 하지마.
설명도 작성 하지마.
코드만 작성해줘.
"""
print(query)


백준 1021, '회전하는 큐' 파이썬으로 풀어줘.
입력 예시는 작성 하지마.
출력 예시도 작성 하지마.
설명도 작성 하지마.
코드만 작성해줘.



In [9]:

solution=run_gpt.run(query)

제공된 solution은 다음과 같습니다.


from collections import deque

n, m = map(int, input().split())
nums = deque(range(1, n + 1))
idxs = list(map(int, input().split()))
ans = 0

for idx in idxs:
    while True:
        if nums[0] == idx:
            nums.popleft()
            break
        else:
            if nums.index(idx) <= len(nums) // 2:
                nums.rotate(-1)
                ans += 1
            else:
                nums.rotate(1)
                ans += 1
                
print(ans)


In [17]:

print(re.sub('    ', '\\t', solution))



from collections import deque

n, m = map(int, input().split())
nums = deque(range(1, n + 1))
idxs = list(map(int, input().split()))
ans = 0

for idx in idxs:
	while True:
		if nums[0] == idx:
			nums.popleft()
			break
		else:
			if nums.index(idx) <= len(nums) // 2:
				nums.rotate(-1)
				ans += 1
			else:
				nums.rotate(1)
				ans += 1
				
print(ans)


In [13]:
solution_lines = solution.split('\n')
solution_lines

['',
 '',
 'from collections import deque',
 '',
 'n, m = map(int, input().split())',
 'nums = deque(range(1, n + 1))',
 'idxs = list(map(int, input().split()))',
 'ans = 0',
 '',
 'for idx in idxs:',
 '    while True:',
 '        if nums[0] == idx:',
 '            nums.popleft()',
 '            break',
 '        else:',
 '            if nums.index(idx) <= len(nums) // 2:',
 '                nums.rotate(-1)',
 '                ans += 1',
 '            else:',
 '                nums.rotate(1)',
 '                ans += 1',
 '                ',
 'print(ans)']

In [15]:
for line in solution_lines:
    line = re.sub('    ','\t', line)
    print(line)



from collections import deque

n, m = map(int, input().split())
nums = deque(range(1, n + 1))
idxs = list(map(int, input().split()))
ans = 0

for idx in idxs:
	while True:
		if nums[0] == idx:
			nums.popleft()
			break
		else:
			if nums.index(idx) <= len(nums) // 2:
				nums.rotate(-1)
				ans += 1
			else:
				nums.rotate(1)
				ans += 1
				
print(ans)


In [6]:
crawler = Crawler(id, pw)
crawler.login_solved()

In [7]:
crawler.summit_solution(number, solution)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

In [2]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
options = webdriver.ChromeOptions()
options.add_argument('user-agent=' + user_agent)

In [3]:
driver = webdriver.Chrome()

In [4]:
driver.get('https://www.acmicpc.net/submit/1064')

In [8]:
last_line = driver.find_elements(By.XPATH, '//*[@id="submit_form"]/div[3]/div/div/div[6]/div[1]/div/div/div/div[5]/div')[-1].text[1:]


'\n    '

In [18]:
import time

In [9]:
import re

In [12]:
result = re.sub('[^ ]', '', last_line)
result, len(result)

('    ', 4)

In [13]:
solution = """\
from collections import deque

n, m = map(int, input().split())
queue = deque([i for i in range(1, n+1)])
nums = list(map(int, input().split()))
count = 0

for num in nums:
    while True:
        if queue[0] == num:
            queue.popleft()
            break
        else:
            if queue.index(num) <= len(queue) // 2:
                queue.rotate(-1)
            else:
                queue.rotate(1)
            count += 1

print(count)
"""

In [14]:
solution_lines = solution.split('\n')

In [19]:
driver.find_element(By.XPATH, '//*[@id="submit_form"]/div[3]/div/div/div[6]').click()
textarea = driver.find_element(By.XPATH, '//*[@id="submit_form"]/div[3]/div/div/div[1]/textarea') 
for line in solution_lines:
    textarea.send_keys(line+'\n')
    last_line = driver.find_elements(By.XPATH, '//*[@id="submit_form"]/div[3]/div/div/div[6]/div[1]/div/div/div/div[5]/div')[-1].text[1:]
    nSpace = len(re.sub('[^ ]', '', last_line))
    for i in range(nSpace):
        textarea.send_keys(Keys.BACKSPACE)


In [23]:
driver.find_element(By.XPATH, '//*[@id="submit_button"]').click()
pattern = '채점'
while True:
    result_text = driver.find_element(By.XPATH, '//*[@id="status-table"]/tbody/tr/td[@class="result"]').text
    if re.search(pattern, result_text):
        print(result_text)
        time.sleep(1)
    else:
        print('결과 : ', result_text)
        break
    

채점 준비 중
채점 중 (3%)
채점 중 (11%)
채점 중 (19%)
채점 중 (27%)
채점 중 (35%)
채점 중 (43%)
채점 중 (51%)
채점 중 (58%)
채점 중 (65%)
채점 중 (73%)
채점 중 (81%)
채점 중 (89%)
채점 중 (97%)
채점 끝
결과 :  맞았습니다!!
